# **CUDA Programming**
#### Creating a kernel that can do vector addition of two different unidimensional vectors/tensors (1D arrays)

In [1]:
import torch, os, math, gzip, pickle
import matplotlib.pyplot as plt
from urllib.request import urlretrieve
from pathlib import Path
import numpy as np
from torch import tensor
import torchvision as tv
import torchvision.transforms.functional as tvf
from torchvision import io
from torch.utils.cpp_extension import load_inline

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [161]:
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [162]:
%pip install -q wurlitzer ninja

In [163]:
%load_ext wurlitzer

The wurlitzer extension is already loaded. To reload it, use:
  %reload_ext wurlitzer


In [164]:
def load_cuda(cuda_src, cpp_src, funcs, opt=False, verbose=False):
    return load_inline(cuda_sources=[cuda_src], cpp_sources=[cpp_src], functions=funcs,
                       extra_cuda_cflags=["-O2"] if opt else [], verbose=verbose, name="inline_ext")

In [165]:
cuda_begin = r'''
#include <torch/extension.h>
#include <stdio.h>
#include <c10/cuda/CUDAException.h>

#define CHECK_CUDA(x) TORCH_CHECK(x.device().is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)

inline unsigned int cdiv(unsigned int a, unsigned int b) { return (a + b - 1) / b;}
'''

In [157]:
cuda_src_mine = cuda_begin + r'''
__global__ void VecAdd_kernel(double* x,double* out, int n) {
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    if (i<n) out[i] = x[i]+x[i+n];
}

torch::Tensor VecAdd(torch::Tensor input) {
    CHECK_INPUT(input);
    int h = input.size(0)/2;
    auto output = torch::empty({h}, input.options());
    int threads = 256;
    VecAdd_kernel<<<cdiv(h,threads), threads>>>(
        input.data_ptr<double>(), output.data_ptr<double>(), h);
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return output;
}'''

In [158]:
cpp_src = "torch::Tensor VecAdd(torch::Tensor input);"

In [3]:
X = torch.tensor(np.random.rand(10000000))  # A 1D tensor of 10000000 random elements
y = torch.tensor(np.random.rand(10000000))  # A 1D tensor of 10000000 random elements
combi = torch.cat((X.flatten(), y.flatten())) # A Concatented 1D tensor of X and y. Contains 20000000 elements in total
combi = combi.contiguous().cuda()   ## Creating a 1D flattened CUDA tensor for Kernelling

In [159]:
module_mine = load_cuda(cuda_src_mine, cpp_src, ['VecAdd'], verbose=True) #loads our cuda kernel in a module

Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
The input conditions for extension module inline_ext have changed. Bumping to version 9 and re-building as inline_ext_v9...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu121/inline_ext/build.ninja...
Building extension module inline_ext_v9...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] c++ -MMD -MF main.o.d -DTORCH_EXTENSION_NAME=inline_ext_v9 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /usr/local/lib/python3.10/dist-packages/torch/include -isystem /usr/local/lib/python3.10/dist-packages/torch/include/torch/csrc/api/include -isystem /usr/local/lib/python3.10/dist-packages/torch/include/TH -isystem /usr/local/lib/python3.10/dist-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /usr/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /root/.cache/torch_extensions/py310_cu121/inline_ext/main.cpp -o main.o 
[2/3] /usr/local/cuda/bin/nvcc  -DTORCH_EXTENSION_NAME=inline_ext_v9 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /usr/local/lib/python3.10/dist-packages/torch/include -isystem /usr/local/lib/python3.10/dist-packages/torch/include/

Loading extension module inline_ext_v9...


In [101]:
dir(module_mine)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'rgb_to_grayscale']

# Doing Vector Addition on X and y using CUDA kernel

In [135]:
%%time
res = module_mine.VecAdd_kernel(combi)
h = res.shape
h

CPU times: user 1.17 ms, sys: 0 ns, total: 1.17 ms
Wall time: 1.18 ms


torch.Size([10000000])

#Doing Vector Addition on X and y using a Python function

In [139]:
%%time
def VecAdd(A,B):
  C=[0]*len(A)
  for i in range(len(A)):
    C[i]=A[i]+B[i]
  return C
Pyres = VecAdd(X,y)

CPU times: user 1min 14s, sys: 2.99 s, total: 1min 17s
Wall time: 1min 17s


In [4]:
%%time
def PytVecAdd(A,B):
  return A+B
PyTorchRes = PytVecAdd(X,y)

CPU times: user 21.8 ms, sys: 37.3 ms, total: 59 ms
Wall time: 62.5 ms


#Writing a function to check if accurate vector addition has been done over all the elements

In [151]:
def AllElemCheck(A,B,C):
  if len(A)!=len(B) or (len(A)!=len(C)):
    print("Dimension Mismatch")
    return False
  else:
    for i in range(len(A)):
      if(A[i]+B[i]!=C[i]):
        print(f"A[i]+B[i]!=C[i]")
        return False
  return True



#Calling our check function to see if the data is accurate. True = Accurate/ False = Inaccurate

In [154]:
AllElemCheck(X,y,res)

True

***Since the result is True there correct vector addition has been implemented***

#The CUDA kernel took 1.17 ms for computation whereas a normal pythonic function took 77 seconds (1m 17s) and a straightforwards PyTorch based method took 59ms. That is 45294x times faster than Python and 34.7x Faster than PyTorch